In [1]:
from pynq import DefaultHierarchy
from pynq import allocate
from math import log
import numpy as np

class FFT_Block_Driver(DefaultHierarchy):
    def __init__(self,description):
        super().__init__(description)
        self.config = 0
        self.fft_size = 0
        self.output_buffer = 0
        
    def _convert_to_data(self, fft_direction, fft_size):
        """
            Craftea los 24bits que se envían al módulo IP FFT para configurarlo. 
            Defaultea a tamaño mas grande configurado con todos los canales en FWD
            Ejemplo de uso: convert_to_data('1',2048)

            Parámetros:
            - fft_direction (int): Un entero que representa la dirección del FFT. (FFT - Señal a Freq, IFFT - Freq a Señal)
            - size (int): El tamaño del FFT en puntos, que debe ser una potencia de 2. Por ejemplo, 1024.

            Retorna:
            - Un entero que representa los datos formateados combinando la dirección del FFT, un byte de ceros y
              el tamaño del FFT como se describió anteriormente.
        """
        # MSB pone el IP como FFT o FFT Inverso. Lo ponemos en 1 al solo haber un canal. zfill padea valores hasta completar 1byte
        fft_direction.zfill(8) # "011" -> 00000011

        # Segundo byte todo 0s
        byte2 = '0'*8

        # Ultimos 5 bits -> FFT size (1024 sería 2^10, entonces convertimos 10 a binario)
        x = int(log(fft_size, 2))
        _size = bin(x)[2:]
        _size.zfill(8)

        # Bytes crafteados
        tdata = fft_direction+byte2+_size
        return int(tdata, 2)

    def configure (self, fft_dir, fft_size):
        """
            Transforma los argumentos y los manda al DMA de configuración

            Parámetros:
            - fft_dir (int): Un entero que representa la dirección del FFT. (FFT - Señal a Freq, IFFT - Freq a Señal)
            - fft_size (int): El tamaño del FFT en puntos, que debe ser una potencia de 2. Por ejemplo, 1024.

            Retorna:
            - Un entero que representa los datos formateados combinando la dirección del FFT, un byte de ceros y
              el tamaño del FFT como se describió anteriormente.
        """
        # Actualizar variables clase
        self.fft_size = fft_size
        self.config = self._convert_to_data(fft_dir, fft_size)
        self.output_buffer = allocate(self.fft_size,np.csingle) # Crear buffer (64b) para mandar datos al DMA, ahorrar tiempo despues
        
        # Crear variable temporal con nuestra configuración
        temp = allocate(1, np.uint32)
        temp[0] = self.config
        
        # Mandar configuración
        self.config_dma.sendchannel.transfer(temp)
        self.config_dma.sendchannel.wait()
        
        # Borrar variable temporal
        del temp
        
        
    def stream_fft(self, input_buffer):
        """
            Ejecuta la función de FFT Acelerada con los datos especificados

            Parámetros:
            - input_buffer (fft_size * np.csingle): Datos a procesar por el fft

            Retorna:
            - Resultado FFT
        """
        # No crear buffer aquí para ahorrar tiempo de cálculo
        
        # Mandar y esperar recibir datos
        self.datos_dma.sendchannel.transfer(input_buffer)
        self.datos_dma.recvchannel.transfer(self.output_buffer)
        self.datos_dma.sendchannel.wait()
        self.datos_dma.recvchannel.wait()
        
        return self.output_buffer
    
    # Reemplazar el DefaultHierarchy para utilizar la clase de driver
    @staticmethod
    def checkhierarchy(description):
        if "datos_dma" in description["ip"] \
            and "config_dma" in description["ip"]:
                    return True
        return False